In [135]:
import numpy as np
import pandas as pd
from neo4j import GraphDatabase
from typing import List, Dict
import os
import streamlit as st
import re
import pandas as pd
import ast

## Processing Raw Data(Scraped from course catalogue)

## Opening and Specifying Directory

In [101]:
source_directory = 'data/raw'
target_directory = 'data/processed'

# Define the source and target directories
def is_graduate_course(title):
    match = re.search(r'\b\d{1,3}', title)  # Extract course number
    if match:
        course_number = int(match.group())  # Convert to integer
        return course_number >= 200  # Check if it's a graduate course
    return False  # If no number is found, consider it not a graduate course

# Split the title and extract the first two elements
def get_index(title):
    split_title = title.split()  # Split the title by spaces
    return " ".join(split_title[:2]).strip(".")  # Join the first two elements back into a string

def get_name(title):
    # Split the title by whitespace
    parts = title.split()
    
    # Remove the first two elements (course code and number)
    course_name_parts = parts[2:-1]  # Exclude the last element (e.g., '(4)')
    
    # Join the remaining parts to reconstruct the course name
    course_name = ' '.join(course_name_parts)
    
    return course_name

def get_unit(description):
    # Use regex to find the number inside parentheses
    match = re.search(r'\((\d+)\)', description)
    if match:
        return int(match.group(1))  # Convert the unit to an integer
    return None  # Return None if no unit is found

def get_tags(description):
    # Split the string by "Tags:" and check if tags exist
    if "Tags:" in description:
        tags_part = description.split("Tags:")[1]  # Extract the part after "Tags:"
        # Split tags into a list and strip any extra whitespace
        tags = [tag.strip() for tag in tags_part.split(',')]
        return tags
    return []  # Return an empty list if no tags are found

## Extracting Key Informations(Prereqs, Units, Major Restrictions)

In [136]:
for file_name in os.listdir(source_directory):
    
    if file_name.endswith('.csv'):  # Only process CSV files
        prerequisites = []
        major_restrictions = []
        course_indexes = []
        course_units = []
        course_titles = []
        course_descriptions = []
        course_tags = []
        source_file_path = os.path.join(source_directory, file_name)
        target_file_path = os.path.join(target_directory, file_name)
        
        # Read the CSV file
        raw_data = pd.read_csv(source_file_path)
        data = raw_data[~raw_data['Title'].apply(is_graduate_course)]

        # Extract course titles into a list
        course_indexes = data['Title'].apply(get_index).tolist()
        course_titles = data['Title'].apply(get_name).tolist()
        course_units = data['Title'].apply(get_unit).tolist()
        course_tags = data['Title'].apply(get_tags).tolist()
        
        
        # Extract descriptions into a list
        course_descriptions = data['Description'].tolist()

        # Loop through the course descriptions to extract key info
        for description in course_descriptions:
            # Check for major restrictions
            if "restricted to" in description.lower():
                # Extract the part mentioning restricted majors
                match = re.search(r"restricted to (.+?)(?:\.|$)", description, re.IGNORECASE)
                if match:
                    restriction_text = match.group(1)
                    # Extract major codes (e.g., CS25, EC26)
                    majors = re.findall(r"[A-Z]{2}\d{2}", restriction_text)
                    
                    # Check if unrestricted condition is also mentioned
                    if "all other students will be allowed" in description.lower():
                        major_restrictions.append([])  # Unrestricted as space permits
                    else:
                        major_restrictions.append(majors)  # Only restricted to listed majors
                else:
                    major_restrictions.append([])  # No valid restriction found
            else:
                major_restrictions.append([])  # No restrictions mentioned
            
            # Match prerequisites that look like course numbers (e.g., "CSE 12", "MATH 10A")
            match = re.search(r"Prerequisite[s]*: (.+?)(?:\.|$)", description)
            if match:
                # Extract the part of the description mentioning prerequisites
                prereqs_text = match.group(1)
                # Use regex to find course numbers (e.g., "MATH 10A", "CSE 12")
                prereqs = re.findall(r"[A-Z]{2,4} \d+[A-Z]?", prereqs_text)
            else:
                prereqs = []  # No prerequisites found
            prerequisites.append(prereqs)
            
        # Display the results in a structured format
        structured_data = {
            "Course_Index": course_indexes,
            "Course_Title": course_titles,
            "Course_Units" : course_units,
            "Course_Description": course_descriptions,
            "Course_Prerequisites": prerequisites,
            "Major_Restriction" : major_restrictions, 
            "Course_Tags": course_tags
        }

        df_structured = pd.DataFrame(structured_data)
        df_structured.to_csv(target_file_path, index=False)
        
        
        print("Sent to", target_file_path)


Exception ignored in: <function Workspace.__del__ at 0x00000248E5DE7CE0>
Traceback (most recent call last):
  File "c:\Users\theso\VSCODE Projects\llm-chatbot-python\.venv\Lib\site-packages\neo4j\_sync\work\workspace.py", line 81, in __del__
    self.close()
  File "c:\Users\theso\VSCODE Projects\llm-chatbot-python\.venv\Lib\site-packages\neo4j\_sync\work\session.py", line 213, in close
    self._connection.fetch_all()
  File "c:\Users\theso\VSCODE Projects\llm-chatbot-python\.venv\Lib\site-packages\neo4j\_sync\io\_bolt.py", line 864, in fetch_all
    detail_delta, summary_delta = self.fetch_message()
                                  ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\theso\VSCODE Projects\llm-chatbot-python\.venv\Lib\site-packages\neo4j\_sync\io\_bolt.py", line 847, in fetch_message
    tag, fields = self.inbox.pop(
                  ^^^^^^^^^^^^^^^
  File "c:\Users\theso\VSCODE Projects\llm-chatbot-python\.venv\Lib\site-packages\neo4j\_sync\io\_common.py", line 72, in pop
    sel

KeyError: 'Title'

In [133]:
class CourseDatabase:
    def __init__(self, uri: str, username: str, password: str):
        """Initialize connection to Neo4j database."""
        self.driver = GraphDatabase.driver(uri, auth=(username, password))

    def close(self):
        """Close the database connection."""
        self.driver.close()

    def add_course(self, code: str, name: str) -> None:
        """Add a new course to the database."""
        with self.driver.session() as session:
            session.execute_write(self._create_course, code, name)

    @staticmethod
    def _create_course(tx, code: str, name: str):
        query = """
        MERGE (c:Course {code: $code})
        SET c.name = $name
        RETURN c
        """
        result = tx.run(query, code=code, name=name)
        return result.single()

    def add_prerequisite(self, course_code: str, prereq_codes: list) -> None:
        """Add a prerequisite relationship between courses."""
        with self.driver.session() as session:
            session.execute_write(self._create_prerequisite, course_code, prereq_codes)

    @staticmethod
    def _create_prerequisite(tx, course_code: str, prereq_codes: list):
        query = """
        MATCH (c:Course {code: $course_code})
        UNWIND $prereq_codes AS prereq_code
        MATCH (p:Course {code: prereq_code})
        MERGE (c)-[:REQUIRES]->(p)
        """
        print("Relationship made", course_code)
        tx.run(query, course_code=course_code, prereq_codes=prereq_codes)

    def get_prerequisites(self, course_code: str, recursive: bool = True) -> List[Dict]:
        """Get prerequisites for a course. If recursive=True, get all indirect prerequisites as well."""
        with self.driver.session() as session:
            return session.execute_read(self._get_prerequisites, course_code, recursive)

    @staticmethod
    def _get_prerequisites(tx, course_code: str, recursive: bool):
        depth = "*" if recursive else "1"
        query = f"""
        MATCH (c:Course {{code: $code}})-[:REQUIRES{depth}]->(p:Course)
        RETURN p.code as code, p.name as name
        """
        result = tx.run(query, code=course_code)
        return [dict(record) for record in result]

    def get_available_courses(self, completed_courses: List[str]) -> List[Dict]:
        """Get courses where all prerequisites have been completed."""
        with self.driver.session() as session:
            return session.execute_read(self._get_available_courses, completed_courses)

    @staticmethod
    def _get_available_courses(tx, completed_courses: List[str]):
        query = """
        MATCH (c:Course)
        WHERE (
            // Either has no prerequisites
            NOT EXISTS((c)-[:REQUIRES]->(:Course))
            OR 
            // Or all prerequisites are completed
            ALL(prereq IN [(c)-[:REQUIRES]->(p) | p.code] 
                WHERE prereq IN $completed_courses)
        )
        // Exclude courses that are already completed
        AND NOT c.code IN $completed_courses
        RETURN c.code as code, c.name as name
        """
        result = tx.run(query, completed_courses=completed_courses)
        return [dict(record) for record in result]

In [4]:
uri = st.secrets["uri"]
username = st.secrets["username"]
password = st.secrets["password"]

db = CourseDatabase(
    uri=uri,
    username=username,
    password=password
)

## Populating the Neo4j Database

In [134]:
source_directory = "data/processed/"
for file_name in os.listdir(source_directory):
    if file_name.endswith('.csv'):
        # Load the CSV file to examine its structure
        file_path = os.path.join(source_directory, file_name)
        print(file_path)
        data = pd.read_csv(file_path)

        courses = list(zip(data['Course_Index'], data['Course_Title']))
        prerequisites = list(zip(data['Course_Index'], data['Course_Prerequisites']))
        for code, name in courses:
            db.add_course(code, name)
        for code, prereq in prerequisites:
            print("Adding prereq", type(prereq))
            db.add_prerequisite(code, ast.literal_eval(prereq))
        print(file_path, "complete")
            

data/processed/CSE Course Data.csv


C:\Users\theso\AppData\Local\Temp\ipykernel_7092\758587648.py:12: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:


BufferError: Existing exports of data: object cannot be re-sized

In [118]:
print("Prerequisites for MATH 154:")
prereqs = db.get_prerequisites("MATH 154")
for prereq in prereqs:
    print(f"- {prereq['code']}: {prereq['name']}")

Prerequisites for MATH 154:


C:\Users\theso\AppData\Local\Temp\ipykernel_7092\758587648.py:41: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:


In [ ]:
# print("Available courses (completed: CS101, MATH101):")
# available = db.get_available_courses(["CS101", "MATH101"])
# for course in available:
#     print(f"- {course['code']}: {course['name']}")

Available courses (completed: CS101, MATH101):
- CS201: Data Structures
- MATH201: Calculus II


In [10]:
db.close()